# Demo

In this notebook we will demonstrate how to use the package for text classification tasks. We will introduce some of the functionality along the way.

## Dataset

We start by producing a dummy dataset to train our model on. The dataset is constructed as follows:

- The dataset has three possible labels "A", "B" and "C".
- Each sample is a short 'text' of length between 5 and 25 tokens over the vocabulary ["a", "b", "c", "d", "e"].
- A text with label "A" can contain any token other than "a" and analogous for "B" and "C".
- For simplicities sake we pad every text to length 25 with the padding token "p" so that the full vocabulary is ["p", "a", "b", "c", "d", "e"].
- We sample 3000 texts of which we will use the first 2500 for training and the last 500 for testing.

In [1]:
import random
import torch

In [2]:
target_names = ["A", "B", "C"]
vocab_nopad = ["a", "b", "c", "d", "e"]
vocab = ["p"] + vocab_nopad
data = [
    {
        "label": (label:=random.choice(target_names)),
        "tokens": random.choices([t for t in vocab_nopad if label.lower() != t], k=(k:=random.randint(5,25))) + ["p"] * (25-k)
    }
    for _ in range(3000)
]

In [3]:
print(data[0])

{'label': 'C', 'tokens': ['a', 'e', 'b', 'e', 'b', 'b', 'a', 'd', 'a', 'd', 'a', 'a', 'e', 'd', 'e', 'a', 'b', 'd', 'b', 'd', 'b', 'p', 'p', 'p', 'p']}


We need to turn our data into tensors that our model can handle.

In [4]:
token2idx = {t: i for i,t in enumerate(vocab)}
label2idx = {l: i for i,l in enumerate(target_names)}

In [5]:
text_tensor = torch.tensor([[token2idx[t] for t in sample["tokens"]] for sample in data], dtype=torch.int64)
label_tensor = torch.tensor([label2idx[sample["label"]]  for sample in data], dtype=torch.int64)

For ease of use we wrap everything in a pytorch dataset.

In [6]:
traindata = torch.utils.data.TensorDataset(text_tensor[:2500], label_tensor[:2500])
testdata = torch.utils.data.TensorDataset(text_tensor[2500:], label_tensor[2500:])

## The model

We build a small SWEM-model to train on the data. The model will have the following structure:

- A worddrop embedding with 6 embeddings (corresponding to the vocab) and an embedding dimension of 3.
- A linear layer of size 3.
- A hierarchical pooling layer with window size 4.
- Another linear layer of size 3 and a final layer of size 3 whose outputs are the class logits for the classification task.

We could construct the model directly from its *\_\_init\_\_*-method but we can also specify the configuration first and let the *from_config*-method do the rest for us.

In [7]:
from swem.models.swem import SwemConfig, Swem

In [8]:
config = SwemConfig.from_dict({
    "embedding": {
        "type": "WordDropEmbedding",
        "num_embeddings": 6,
        "embedding_dim": 3,
        "padding_idx": 0,
        "p": 0.2
    },
    "pooling": {
        "type": "HierarchicalPooling",
        "window_size": 4
    },
    "pre_pooling_dims": (3, ),
    "post_pooling_dims": (3, 3),
    "dropout": 0.2
})

In [9]:
model = Swem.from_config(config)

In [10]:
model

Swem(
  (embedding): WordDropEmbedding(6, 3, padding_idx=0)
  (pooling_layer): HierarchicalPooling(
    (avg_pooling): AvgPool2d(kernel_size=(4, 1), stride=1, padding=0)
  )
  (pre_pooling_trafo): Sequential(
    (0): Linear(in_features=3, out_features=3, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
  )
  (post_pooling_trafo): Sequential(
    (0): Linear(in_features=3, out_features=3, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=3, out_features=3, bias=True)
  )
)

## Training

We train the model for 20 epochs with a batch size of 8 and an Adam optimizer with learning rate 3e-4 (Karpathy's constant 😉). If a GPU is available we use it otherwise we default to the CPU. We use the *to_device*-function to transfer the tokens and labels to the specified device at once.

In [11]:
from swem.utils.torch_utils import to_device

In [12]:
epochs = 20
batch_size = 8
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
loss_fn = torch.nn.CrossEntropyLoss()
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [13]:
train_dataloader = torch.utils.data.DataLoader(
    traindata,
    batch_size=batch_size,
    shuffle=True
)

for i in range(epochs):
    print(f"Starting epoch {i+1}")
    model.train()
    for batch in train_dataloader:
        batch = to_device(batch, device=device)
        tokens, labels = batch
        output = model(tokens)
        loss = loss_fn(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Starting epoch 20


## Evaluation

Let us now evaluate our model on the testset. For that purpose we iterate batchwise over the testset and aggregate the metrics along the way. This is achieved by making use of the *ClassificationReport*.

In [14]:
from swem.metrics import ClassificationReport

In [15]:
report = ClassificationReport(target_names=target_names)

In [16]:
test_dataloader = torch.utils.data.DataLoader(
    testdata,
    batch_size=batch_size,
    shuffle=False
)

model.eval()
for batch in test_dataloader:
    with torch.no_grad():
        batch = to_device(batch, device=device)
        tokens, labels = batch
        logits = model(tokens)
        report.update(logits, labels)
    

In [17]:
report

{
  "num_samples": 500,
  "accuracy": 0.644,
  "class_metrics": {
    "A": {
      "support": 168,
      "recall": 1.0,
      "precision": 0.9230769230769231,
      "f1_score": 0.9600000000000001
    },
    "B": {
      "support": 160,
      "recall": 0.9375,
      "precision": 0.4838709677419355,
      "f1_score": 0.6382978723404255
    },
    "C": {
      "support": 172,
      "recall": 0.023255813953488372,
      "precision": 0.5,
      "f1_score": 0.04444444444444444
    }
  }
}

## Saving the model

Let us save our model to disk for later use.

In [18]:
from pathlib import Path

In [19]:
model_path = Path("./model")

In [20]:
model.save(model_path)

The *.save*-method save both the config and the weights in the directory:

In [21]:
print(list(model_path.iterdir()))

[PosixPath('model/weights.pt'), PosixPath('model/config.json')]


If we want to use the model later on we can simply load it,

In [22]:
model_loaded = Swem.load(model_path)

Let's make sure config and weights of the loaded model are actually the same as for the original model.

In [23]:
model_loaded.config == model.config

True

In [24]:
all(
    torch.allclose(model_param, model_loaded.state_dict()[param_name]) 
    for param_name, model_param in model.state_dict().items()
)

True